In [56]:
import pandas as pd
import numpy as np

In [57]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [58]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [59]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [60]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [61]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000003,6.5,1839
3,tt0000004,5.5,178
4,tt0000005,6.2,2624


# Preprocessing |  AKAS

In [62]:
akas = akas[akas['region'] == 'US']

In [63]:
#Replace "\N" with np.nan
akas.replace({'\\N': np.nan}, inplace=True)

# Preprocessing | TITLE BASICS

In [64]:
#Replace "\N" with np.nan
basics.replace({'\\N': np.nan}, inplace=True)
basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [65]:
#Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace=True)
basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [66]:
#Eliminate movies that are null for genre
basics.dropna(subset=['genres'], inplace=True)
basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [67]:
#keep only titleType==Movie
basics = basics[basics['titleType'] == 'movie']
basics.head(5)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,NaN,100,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"


In [68]:
#delete the NaNs
basics = basics.dropna(subset=['startYear'])

# count the NaNs
basics['startYear'].isna().sum()

0

In [69]:
# Convert 'startYear' column to integer
basics['startYear'] = basics['startYear'].astype(int)

In [70]:
#keep startYear 2000-2022
basics = basics [(basics ['startYear'] >= 2000) & (basics['startYear'] <= 2022)]


In [71]:
#Eliminate movies that include "Documentary" in genre (see tip below)
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics  =basics [~is_documentary]

In [72]:
#Keep only US movies (Use AKAs table, see "Filtering one dataframe based on another" section below)
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34803       True
42384       True
61115       True
67667       True
86799       True
           ...  
9974130     True
9974139     True
9974178    False
9974223     True
9974307    False
Name: tconst, Length: 147840, dtype: bool

In [73]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67667,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86799,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9973595,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9973990,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9974130,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9974139,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


# Preprocessing |  RATINGS

In [74]:
#Replace "\N" with np.nan
ratings.replace({'\\N': np.nan}, inplace=True)

In [75]:
# Filter the ratings table down to only include the US by using the filter akas dataframe
keepers =ratings['tconst'].isin(akas['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1325576    False
1325577    False
1325578    False
1325579    False
1325580    False
Name: tconst, Length: 1325581, dtype: bool

In [76]:
ratings = ratings[keepers]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
4,tt0000005,6.2,2624
5,tt0000006,5.1,182
6,tt0000007,5.4,821
...,...,...,...
1325542,tt9916200,8.1,230
1325543,tt9916204,8.2,264
1325550,tt9916348,8.3,18
1325551,tt9916362,6.4,5391


# Check files

In [77]:
akas.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1448208 entries, 5 to 36400744
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1448208 non-null  object
 1   ordering         1448208 non-null  int64 
 2   title            1448208 non-null  object
 3   region           1448208 non-null  object
 4   language         3975 non-null     object
 5   types            980689 non-null   object
 6   attributes       46873 non-null    object
 7   isOriginalTitle  1446866 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.4+ MB


In [78]:
basics.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86865 entries, 34803 to 9974223
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          86865 non-null  object
 1   titleType       86865 non-null  object
 2   primaryTitle    86865 non-null  object
 3   originalTitle   86865 non-null  object
 4   isAdult         86865 non-null  object
 5   startYear       86865 non-null  int64 
 6   endYear         0 non-null      object
 7   runtimeMinutes  86865 non-null  object
 8   genres          86865 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.6+ MB


In [79]:
ratings.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 502443 entries, 0 to 1325556
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         502443 non-null  object 
 1   averageRating  502443 non-null  float64
 2   numVotes       502443 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.3+ MB


# Saving Compressed csv.gz Files

In [80]:
# example making new folder with os
import os
os.makedirs('DataProject3/',exist_ok=True) 
# Confirm folder created
os.listdir("DataProject3/")

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

In [81]:
## Save current dataframe to file.
basics.to_csv("DataProject3/title_basics.csv.gz",compression='gzip',index=False)

In [82]:
# Open saved file and preview again
basics = pd.read_csv("DataProject3/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013,NaN,120,"Drama,History"
2,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [83]:
## Save current dataframe to file.
akas.to_csv("DataProject3/title_akas.csv.gz",compression='gzip',index=False)

In [84]:
# Open saved file and preview again
akas = pd.read_csv("DataProject3/title_akas.csv.gz", low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [85]:
## Save current dataframe to file.
ratings.to_csv("DataProject3/title_ratings.csv.gz",compression='gzip',index=False)

In [86]:
# Open saved file and preview again
ratings = pd.read_csv("DataProject3/title_ratings.csv.gz", low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1982
1,tt0000002,5.8,265
2,tt0000005,6.2,2624
3,tt0000006,5.1,182
4,tt0000007,5.4,821
